# ANÁLISIS ESTRATÉGICO DEL MERCADO DE VEHÍCULOS USADOS (EE.UU.)

Este proyecto desarrolla una herramienta analítica para explorar el mercado de anuncios de vehículos usados. El objetivo principal es transformar datos brutos en una aplicación interactiva que permita identificar tendencias de precios, el impacto del kilometraje en el valor de los activos y la distribución de la oferta según la condición de las unidades.

A través de un enfoque de ingeniería de software para datos, este análisis no solo limpia la información, sino que la prepara para un despliegue escalable en la web utilizando **Streamlit**.

---

### Resumen del Dataset
El conjunto de datos contiene detalles de anuncios de venta que incluyen el precio, años del modelo, tipo de combustible, kilometraje (odómetro) y estado físico del vehículo.

---

### Metodología de Trabajo
1. **Configuración del Entorno y Preprocesamiento:** Importación de librerías, carga optimizada, inspección inicial y tratamiento de datos ausentes mediante imputación lógica basada en perfiles de vehículos similares.
2. **Análisis Exploratorio (EDA) y Visualización:** Identificación de correlaciones clave y detección de valores atípicos (*outliers*) de mercado. Diseño y validación de componentes gráficos para la toma de decisiones estratégicas.

---
---

## 1. Configuración del Entorno y Preprocesamiento

En esta sección, inicializamos las herramientas necesarias y cargamos el dataset utilizando el motor de **PyArrow** para maximizar la eficiencia en la lectura. Realizaremos una inspección inicial para detectar inconsistencias y aplicaremos lógica de negocio para el tratamiento de los datos.

In [1]:
# importación de librerías
import pandas as pd
import plotly.express as px
import numpy as np
# Configuración visual y pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# Carga de datos con Pandas 3.0 + PyArrow Backend
# Esto optimiza el uso de memoria y velocidad de ejecución
try:
    df = pd.read_csv('../data/vehicles_us.csv', engine='pyarrow', dtype_backend='pyarrow')
    print("Dataset cargado exitosamente.")
except Exception:
    df = pd.read_csv('vehicles_us.csv', engine='pyarrow', dtype_backend='pyarrow')
    print("Dataset cargado desde la raíz local.")

Dataset cargado exitosamente.


In [3]:
def inspeccion_inicial(df):
    """Inspección inicial del dataset: info, sample, nulos, duplicados y valores únicos."""
    print('--- INSPECCIÓN DE LOS DATOS ---\n')
    n, p = df.shape[0], df.shape[1]
    print(f"Dimensiones: {n:,} filas, {p} columnas\n")

    print("\n--- Tipos de Datos y Nulos ---")
    df.info(show_counts=True, memory_usage="deep")

    print("\n--- Muestra de los datos ---")
    display(df.sample(n=min(len(df), 5), random_state=42))

    print("--- Nulos por columna ---")
    nulos = df.isnull().sum()
    nulos = nulos[nulos > 0].sort_values(ascending=False)
    if nulos.empty:
        print("Ningún nulo.")
    else:
        reporte = pd.DataFrame({"nulos": nulos, "%": (nulos / len(df) * 100).round(2)})
        display(reporte)

    print("\n--- Duplicados explícitos ---")
    n_dup = df.duplicated().sum()
    print(f"Filas duplicadas: {n_dup}")
    if n_dup > 0:
        display(df[df.duplicated(keep=False)].sort_values(by=list(df.columns)).head(10))

    print("\n--- Valores únicos por columna ---")
    unicos = df.nunique().to_frame("n_unicos")
    unicos["% no nulos"] = (df.count() / len(df) * 100).round(1)
    display(unicos)

inspeccion_inicial(df)

--- INSPECCIÓN DE LOS DATOS ---

Dimensiones: 51,525 filas, 13 columnas


--- Tipos de Datos y Nulos ---
<class 'pandas.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype               
---  ------        --------------  -----               
 0   price         51525 non-null  int64[pyarrow]      
 1   model_year    47906 non-null  double[pyarrow]     
 2   model         51525 non-null  string[pyarrow]     
 3   condition     51525 non-null  string[pyarrow]     
 4   cylinders     46265 non-null  int64[pyarrow]      
 5   fuel          51525 non-null  string[pyarrow]     
 6   odometer      43633 non-null  double[pyarrow]     
 7   transmission  51525 non-null  string[pyarrow]     
 8   type          51525 non-null  string[pyarrow]     
 9   paint_color   42258 non-null  string[pyarrow]     
 10  is_4wd        25572 non-null  double[pyarrow]     
 11  date_posted   51525 non-null  date32[day][pyarrow]
 12  days_lis

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
14229,11995,"2,013.00",chevrolet impala,good,6,gas,<NA>,automatic,sedan,white,<NA>,2018-09-02,77
7481,6995,"2,005.00",ram 1500,excellent,8,gas,"144,518.00",automatic,pickup,blue,1.00,2019-01-01,1
37294,5950,"2,013.00",chevrolet cruze,excellent,4,gas,"107,000.00",manual,sedan,silver,<NA>,2018-08-09,124
21193,1750,"2,004.00",hyundai elantra,excellent,4,gas,"181,256.00",automatic,sedan,silver,<NA>,2018-10-31,67
16857,23900,"2,014.00",chevrolet silverado 1500 crew,good,8,gas,"91,844.00",automatic,pickup,white,1.00,2019-02-15,26


--- Nulos por columna ---


,nulos,%
is_4wd,25953,50.37
paint_color,9267,17.99
odometer,7892,15.32
cylinders,5260,10.21
model_year,3619,7.02



--- Duplicados explícitos ---
Filas duplicadas: 0

--- Valores únicos por columna ---


,n_unicos,% no nulos
price,3443,100.00
model_year,68,93.00
model,100,100.00
condition,6,100.00
cylinders,7,89.80
fuel,5,100.00
odometer,17762,84.70
transmission,3,100.00
type,13,100.00
paint_color,12,82.00


In [4]:
# Analizando la dispersión y tendencia central de 'price'
print("Análisis Estadístico de Precios:")
print(df['price'].agg(['mean', 'median', 'std', 'var']))
# Cuantificar el sesgo
skewness = df['price'].skew()
print(f"\nCoeficiente de asimetría (Skewness): {skewness:.2f}")

Análisis Estadístico de Precios:
mean          12,132.46
median         9,000.00
std           10,040.80
var      100,817,725.19
Name: price, dtype: float64

Coeficiente de asimetría (Skewness): 3.59


### Hallazgos de la Inspección y Estrategia de Limpieza

Tras la inspección inicial del dataset, se han definido estrategias específicas para garantizar la integridad estadística y la eficiencia computacional de la aplicación:

1. **Tratamiento de Valores Ausentes mediante Lógica de Negocio:**
    * **`is_4wd` (Inferencia Booleana):** Se identifica que los valores nulos actúan como un marcador negativo. Se imputarán con `0` (Falso) para transformar la columna en un tipo booleano/entero eficiente.
    * **`model_year` y `cylinders` (Imputación Contextual):** En lugar de usar una mediana global que sesgaría los datos, se utilizará la mediana agrupada por `model`. Esto asegura que un sedán compacto no reciba accidentalmente el número de cilindros de un camión pesado.
    * **`odometer` (Proxy de Desgaste):** Dado que el kilometraje está altamente correlacionado con la edad del vehículo, se imputarán los faltantes utilizando la mediana agrupada por `model_year`.
    * **`paint_color` (Preservación de Categorías):** Se etiquetarán como `unknown` para evitar sesgar el análisis de preferencias de color del mercado.

2. **Optimización de Memoria y Tipado (Pandas 3.0 + PyArrow):**
    * Se realizará un *casting* de tipos hacia los formatos más pequeños posibles (`int8`, `int16`) utilizando el backend de **PyArrow**. Esto es crítico para la velocidad de respuesta de la aplicación Streamlit al filtrar grandes volúmenes de datos.

3. **Gestión de Outliers en Precios:**
    * Se observa una distribución con sesgo a la derecha. Para las visualizaciones de distribución, se aplicará un filtro basado en el percentil 97 para eliminar el ruido de precios atípicos o errores de carga, manteniendo la integridad para el análisis de correlaciones.

### **Ingeniería de Datos y Preprocesamiento**

Basado en la evidencia estadística anterior (sesgo positivo y alta dispersión), procederemos con una limpieza técnica que priorice la integridad del contexto de cada vehículo. 

Las acciones clave incluyen:
* **Imputación Contextual:** Rellenar `model_year`, `cylinders` y `odometer` utilizando medianas agrupadas por modelo o año, evitando el sesgo de las medianas globales.
* **Normalización Booleana:** Transformar `is_4wd` asumiendo que los valores ausentes indican la falta de tracción total.
* **Casting de Tipos:** Optimizar la memoria convirtiendo a tipos enteros de PyArrow (`int8`, `int16`, `int64`), lo que garantiza que la aplicación sea escalable y rápida.

In [5]:
# Tratamiento de is_4wd: Inferencia booleana (NaN -> 0 = no tracción 4WD)
# clip(0,1): normaliza a 0/1 por si hubiera otros valores.
df["is_4wd"] = df["is_4wd"].fillna(0).clip(0, 1).astype("int8[pyarrow]")

# Imputación Contextual: Rellenar nulos basados en grupos de referencia
# Imputamos años y cilindros por Modelo
df['model_year'] = df['model_year'].fillna(
    df.groupby('model')['model_year'].transform('median')
)
df['cylinders'] = df['cylinders'].fillna(
    df.groupby('model')['cylinders'].transform('median')
)

# Imputamos el odómetro por Año del Modelo (Correlación desgaste/antigüedad)
df['odometer'] = df['odometer'].fillna(
    df.groupby('model_year')['odometer'].transform('median')
)

# Manejo de etiquetas y nulos remanentes
df['paint_color'] = df['paint_color'].fillna('unknown')

# Fallback de seguridad: Si un modelo es único y tiene nulos, usamos la mediana global
df['model_year'] = df['model_year'].fillna(df['model_year'].median())
df['cylinders'] = df['cylinders'].fillna(df['cylinders'].median())
df['odometer'] = df['odometer'].fillna(df['odometer'].median())

# Optimización final de tipos: redondear a entero antes del cast (las medianas devuelven float)
df['model_year'] = df['model_year'].round(0).astype('int16[pyarrow]')
df['cylinders'] = df['cylinders'].round(0).astype('int8[pyarrow]')
df['odometer'] = df['odometer'].round(0).astype('int64[pyarrow]')
df['date_posted'] = pd.to_datetime(df['date_posted'])

# Verificación de integridad final
print("Reporte de Calidad de Datos (Valores Nulos):")
print(df.isnull().sum())
print("\nEstructura de memoria optimizada")
print(df.info())

Reporte de Calidad de Datos (Valores Nulos):
price           0
model_year      0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
type            0
paint_color     0
is_4wd          0
date_posted     0
days_listed     0
dtype: int64

Estructura de memoria optimizada
<class 'pandas.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype          
---  ------        --------------  -----          
 0   price         51525 non-null  int64[pyarrow] 
 1   model_year    51525 non-null  int16[pyarrow] 
 2   model         51525 non-null  string[pyarrow]
 3   condition     51525 non-null  string[pyarrow]
 4   cylinders     51525 non-null  int8[pyarrow]  
 5   fuel          51525 non-null  string[pyarrow]
 6   odometer      51525 non-null  int64[pyarrow] 
 7   transmission  51525 non-null  string[pyarrow]
 8   type          51525 non-null  string[pyarrow]
 9   paint_colo

Ahora nuestros datos están listos y en condiciones para su análisis.

---
---

## 2. Análisis Exploratorio de Datos (EDA)

Con un dataset optimizado y libre de valores ausentes, procederemos a explorar las dinámicas del mercado. El objetivo de esta fase no es solo generar gráficos, sino validar hipótesis de negocio y entender los factores que dictan el valor de reventa de un vehículo.

En esta etapa buscaremos responder:
1. **¿Cómo influye la condición del vehículo en la distribución de precios?** (Validación del sesgo identificado).
2. **¿Cuál es la relación entre el kilometraje y el precio?** (Análisis de depreciación).
3. **¿Qué tipos de vehículos y configuraciones dominan el inventario?** (Análisis de nichos).

---

### 2.1. Análisis de Precios y Segmentación por Condición

Anteriormente identificamos un fuerte sesgo a la derecha en los precios. En esta subsección, visualizaremos la distribución segmentada para entender si los valores atípicos pertenecen a categorías específicas (ej. vehículos "new" o "like new").

In [13]:
# Definir el filtro de percentil 97 basado en la inspección previa
# Esto nos permite ver el "corazón" del mercado sin el ruido de los extremos
p97 = df['price'].quantile(0.97)
df_plot = df[df['price'] <= p97]

# Crear histograma interactivo segmentado
fig_price_cond = px.histogram(
    df_plot, 
    x="price", 
    color="condition",
    nbins=100,
    title="Distribución de Precios según la Condición del Vehículo (Hasta P97)",
    labels={'price': 'Precio ($)', 'condition': 'Condición'},
    template="plotly_white",
    barmode='overlay' # Superponer para comparar densidades
)

fig_price_cond.update_layout(
    xaxis_title="Precio de Venta ($)",
    yaxis_title="Frecuencia de Anuncios",
    legend_title="Estado del Vehículo"
)

fig_price_cond.show()

# Insight Rápido: Cálculo de precio mediano por condición
print("Precio Mediano por Condición:")
print(df.groupby('condition')['price'].median().sort_values(ascending=False))

# Creamos una figura con un Boxplot para ver "los outliers en acción"
fig_box = px.box(
    df_plot,
    x="price", 
    color="condition",
    title="Análisis de Outliers: Distribución Total de Precios por Condición",
    labels={'price': 'Precio ($)', 'condition': 'Condición'},
    template="plotly_white"
)

fig_box.update_layout(xaxis_title="Precio de Venta ($)")
fig_box.show()

# Histogramas comparativos: Con y Sin Filtro para Storytelling
print("Comparativa de escala:")
print(f"Precio Máximo: ${df['price'].max():,.2f}")
print(f"Precio Percentil 97: ${df['price'].quantile(0.97):,.2f}")

Precio Mediano por Condición:
condition
new         21,999.00
like new    13,995.00
excellent   10,495.00
good         7,900.00
fair         2,500.00
salvage      2,500.00
Name: price, dtype: double[pyarrow]


Comparativa de escala:
Precio Máximo: $375,000.00
Precio Percentil 97: $34,950.00


**Insights de Mercado: Condición y Volumen**

Tras analizar las distribuciones y el conteo de frecuencias, se extraen las siguientes conclusiones estratégicas:

* **Dominio del Mercado Medio:** El inventario está fuertemente concentrado en las categorías **Excellent**, **Good** y **fair**. Esto sugiere que la plataforma es un mercado de vehículos funcionales de uso diario, no un nicho de coleccionistas (`new`) ni de desguace (`salvage`).
* **Premium de Condición:** Existe un salto de precio mediano significativo entre categorías:
    * De **Good** (\$7,900) a **Excellent** (\$10,495) hay un incremento del **~32%** en el valor.
    * De **Excellent** a **Like New** (\$13,995) el salto es de otro **~33%**.
* **Escasez en los Extremos:** La bajísima frecuencia de anuncios `new` y `salvage` (menos de 10 anuncios en picos máximos) indica que cualquier análisis sobre estas categorías será estadísticamente poco representativo y debe tomarse con cautela.
* **Relación Volumen-Precio:** Se confirma la ley de oferta: a menor precio, mayor volumen de anuncios, concentrándose la mayor masa crítica por debajo de los \$15,000.

---

### 2.2. Análisis de Depreciación: Impacto del Kilometraje en el Valor

La depreciación es el costo oculto más significativo en la propiedad de un vehículo. En esta sección, analizamos la "velocidad" con la que el mercado castiga el precio a medida que aumenta el kilometraje (**odometer**).

**Objetivos del Análisis:**
* **Visualizar la curva de depreciación:** Identificar si la caída de precio es lineal o si existen puntos de quiebre (ej. después de las 100,000 millas).
* **Evaluar la Resiliencia por Condición:** Determinar si un vehículo en estado "Excellent" mantiene un premium de precio constante sobre uno "Good" a pesar de tener un kilometraje elevado.
* **Densidad de Mercado:** Observar en qué rangos de millaje se concentra la mayor oferta de vehículos usados.

Utilizaremos un gráfico de dispersión con **transparencia** para gestionar el solapamiento de más de 50,000 puntos y añadiremos líneas de tendencia (**OLS**) para cuantificar la depreciación.

In [24]:
# Usamos df_plot (filtro P97). Visualizaciones agregadas para leer la depreciación sin ruido.
# Bins de kilometraje (cada 25k millas) para curvas de depreciación
max_miles = min(df_plot["odometer"].max(), 300_000)
bin_edges = np.arange(0, max_miles + 1, 20000)
df_binned = df_plot.copy()
df_binned["odometer_bin"] = pd.cut(df_plot["odometer"], bins=bin_edges, include_lowest=True)

# Precio mediano por bin y condición → curva de depreciación clara
depre = df_binned.groupby(["odometer_bin", "condition"], observed=True)["price"].median().reset_index()
depre["odometer_mid"] = depre["odometer_bin"].apply(lambda b: b.mid)

_config = {"displayModeBar": False, "staticPlot": True}
color_map = {"excellent": "#2F6ECE", "good": "#F1A139", "like new": "#32A852", "fair": "#A83232", "new": "#6B2FCE", "salvage": "#333"}

fig_depre = px.line(
    depre,
    x="odometer_mid",
    y="price",
    color="condition",
    markers=True,
    title="Curva de depreciación: precio mediano por kilometraje y condición",
    labels={"odometer_mid": "Kilometraje (millas)", "price": "Precio mediano ($)", "condition": "Condición"},
    template="plotly_white",
    color_discrete_map=color_map
)
fig_depre.update_layout(
    xaxis_title="Kilometraje (millas)",
    yaxis_title="Precio mediano ($)",
    legend_title="Condición",
    height=520,
    xaxis_range=[0, 300_000],
    xaxis_dtick=20000
)
fig_depre.show(config=_config)

# Densidad de oferta: en qué rangos de millaje se concentra el mercado
# Segunda gráfica: más bins (cada 10k millas) para ver mejor la concentración
bin_edges_dens = np.arange(0, max_miles + 1, 10_000)
df_dens = df_plot.copy()
df_dens["odometer_bin_dens"] = pd.cut(df_plot["odometer"], bins=bin_edges_dens, include_lowest=True)
count_by_bin = df_dens.groupby("odometer_bin_dens", observed=True).size().reset_index(name="anuncios")
count_by_bin["odometer_mid"] = count_by_bin["odometer_bin_dens"].apply(lambda b: b.mid)
fig_dens = px.bar(
    count_by_bin,
    x="odometer_mid",
    y="anuncios",
    title="Concentración de oferta por kilometraje",
    labels={"odometer_mid": "Kilometraje (millas)", "anuncios": "Nº de anuncios"},
    template="plotly_white"
)
fig_dens.update_layout(height=400, showlegend=False, xaxis_dtick=20000)
fig_dens.show(config=_config)

correlation = df["price"].corr(df["odometer"])
print(f"Coeficiente de correlación de Pearson (Precio vs Odómetro): {correlation:.2f}")

Coeficiente de correlación de Pearson (Precio vs Odómetro): -0.42


**Síntesis Estratégica: ¿Qué nos dicen los datos?**

Tras analizar las curvas de depreciación y la densidad de la oferta, se extraen conclusiones críticas que impactan la interpretación del negocio:

**Anomalías y Calidad de Datos (Data Quality)**
* **La paradoja de los "New" con kilometraje:** Se observa que vehículos etiquetados como `new` presentan odómetros de hasta 110,000 millas. Esto sugiere un error de etiquetado en origen (posible confusión entre "modelo del año" y "estado físico"). 
* **El fenómeno de los "Salvage" de lujo:** La valoración positiva en vehículos de salvamento (hasta \$125,000) indica la presencia de activos de alta gama (ej. superdeportivos siniestrados) que conservan un valor residual inmenso, o errores de entrada de datos que actúan como *outliers* extremos.

**Dinámicas de Depreciación por Segmento**
* **El "Muro" de las 100k Millas:** Los vehículos en condición `good` sufren su mayor castigo de valor entre las 50k y 110k millas, perdiendo casi el **60% de su valor** (\$18k → \$7.5k). Este es el punto crítico de decisión para un vendedor.
* **Resiliencia en "Like New":** A diferencia de otros segmentos, los vehículos `like new` muestran una depreciación mucho más lineal y proporcional, siendo el segmento más predecible para modelos de valoración.

**Análisis de Volumen y Correlación**
* **El "Sweet Spot" del Inventario:** La mayor concentración de anuncios se encuentra en el rango de **100k a 160k millas**. Esto define a la plataforma como un mercado de "segunda vida", donde los dueños originales buscan salir del activo antes de que el mantenimiento preventivo mayor sea necesario.
* **Fuerza de la Relación:** El coeficiente de Pearson de **-0.42** confirma una relación inversa moderada. Esto indica que, si bien el kilometraje importa, no es el único factor determinante; variables como el tipo de vehículo (SUV vs. Truck) y la marca tienen un peso específico que "suaviza" el impacto del odómetro.

---

### 2.3. Análisis de Nichos: Composición del Inventario y Valor por Segmento

Para concluir el EDA, analizaremos qué tipos de vehículos dominan la oferta y cómo varía el valor de mercado entre ellos. Esto nos permitirá identificar qué segmentos son los "motores" de la plataforma.

**Preguntas clave:**
* ¿Cuáles son los 3 tipos de vehículos más anunciados?
* ¿Existe una correlación entre el tipo de vehículo y el precio mediano? (Ej: ¿Los Trucks mantienen un valor significativamente más alto?).
* ¿Cómo se distribuye la tracción 4WD en los segmentos líderes?

In [32]:
# 2.3.1 ¿Cuáles son los 3 tipos de vehículos más anunciados?
_config = {"displayModeBar": False, "staticPlot": True}
count_by_type = df["type"].value_counts().reset_index()
count_by_type.columns = ["type", "anuncios"]
count_by_type = count_by_type.sort_values("anuncios", ascending=True)
fig_top = px.bar(
    count_by_type,
    x="anuncios",
    y="type",
    orientation="h",
    title="Tipos de vehículos más anunciados",
    labels={"anuncios": "Nº de anuncios", "type": "Tipo de vehículo"},
    template="plotly_white",
    color="anuncios",
    color_continuous_scale=[[0, "#3B82F6"], [1, "#1E40AF"]]
)
fig_top.update_layout(height=380, showlegend=False, coloraxis_showscale=False)
fig_top.show(config=_config)
top3 = count_by_type.tail(3)["type"].tolist()[::-1]
print("Top 3 tipos más anunciados:", top3)

Top 3 tipos más anunciados: ['SUV', 'truck', 'sedan']


In [33]:
# ¿Correlación tipo de vehículo y precio mediano? (¿Los trucks mantienen más valor?)
median_by_type = df.groupby("type", observed=True)["price"].median().sort_values(ascending=True).reset_index()
median_by_type.columns = ["type", "precio_mediano"]
fig_median = px.bar(
    median_by_type,
    x="precio_mediano",
    y="type",
    orientation="h",
    title="Precio mediano por tipo de vehículo",
    labels={"precio_mediano": "Precio mediano ($)", "type": "Tipo de vehículo"},
    template="plotly_white",
    color="precio_mediano",
    color_continuous_scale=[[0, "#3B82F6"], [1, "#1E40AF"]]
)
fig_median.update_layout(height=380, showlegend=False, coloraxis_showscale=False)
fig_median.show(config=_config)

In [34]:
# ¿Distribución de tracción 4WD en los segmentos líderes?
count_by_type = df["type"].value_counts().reset_index()
count_by_type.columns = ["type", "anuncios"]
top_types = count_by_type.nlargest(5, "anuncios")["type"].tolist()
df_top = df[df["type"].isin(top_types)]
pct_4wd = df_top.groupby("type", observed=True).agg(
    total=("is_4wd", "count"),
    con_4wd=("is_4wd", "sum")
).assign(pct_4wd=lambda x: (x["con_4wd"] / x["total"] * 100).round(1)).reset_index()
pct_4wd = pct_4wd.sort_values("pct_4wd", ascending=True)
fig_4wd = px.bar(
    pct_4wd,
    x="pct_4wd",
    y="type",
    orientation="h",
    title="% de vehículos con tracción 4WD en los 5 tipos más anunciados",
    labels={"pct_4wd": "% con 4WD", "type": "Tipo de vehículo"},
    template="plotly_white",
    text="pct_4wd",
    color="pct_4wd",
    color_continuous_scale=[[0, "#3B82F6"], [1, "#1E40AF"]]
)
fig_4wd.update_traces(texttemplate="%{text:.1f}%", textposition="outside")
fig_4wd.update_layout(height=320, showlegend=False, coloraxis_showscale=False)
fig_4wd.show(config=_config)

**Síntesis de Nichos: Dominio de Segmentos y Configuración Técnica**

El análisis de la composición del inventario revela una segmentación clara del mercado, donde la utilidad y el valor de reventa dictan las reglas del juego.

**Estructura del Inventario (El "Big Three")**
* El catálogo está dominado por tres categorías que representan la gran mayoría de la oferta: **SUV, Truck y Sedan**. 
    * Mientras que SUVs y Trucks lideran con más de 12,000 anuncios cada uno, los Sedanes mantienen una presencia sólida (~12k). 
    * Existe una caída drástica en el volumen hacia segmentos de nicho como **Coupe** (~2,300) o **Hatchback**, lo que indica que la plataforma se especializa en vehículos de gran tamaño o movilidad familiar estándar.

**Jerarquía de Precios y Retención de Valor**
* Los datos muestran una correlación directa entre el tipo de vehículo y su posicionamiento de precio:
    * **Líderes de Valor:** Los **Trucks y Pickups** comandan los precios medianos más altos (entre **$13k y $15k**). Esto confirma que son activos con alta retención de valor debido a su demanda para trabajo y durabilidad.
    * **El Fenómeno "Coupe":** A pesar de su bajo volumen, los Coupes alcanzan precios medianos altos, sugiriendo un segmento de vehículos deportivos o recreativos con un posicionamiento *premium*.
    * **Segmentos de Entrada:** Los **Sedan, Hatchback y Minivan** se posicionan como las opciones más económicas ($6k - $7k), actuando como la puerta de entrada para compradores con presupuesto limitado.

**Especialización Técnica (4WD)**
* La tracción 4WD no es una opción estética, es una necesidad funcional en los segmentos líderes:
    * En **Trucks y Pickups**, la presencia de 4WD es casi universal (llegando a ser el estándar del segmento).
    * En contraste, en **Sedanes y Coupes**, esta característica es casi inexistente (~4%). 
    * **Insight Estratégico:** El inventario está fuertemente inclinado hacia vehículos con capacidades *off-road* o climas adversos, lo que define el perfil geográfico y de uso del comprador promedio de la plataforma.

---
---

## 3. Conclusiones Generales

Tras finalizar el análisis exploratorio y el preprocesamiento técnico del inventario de vehículos, se consolidan los siguientes pilares estratégicos:

* **Eficiencia Técnica y Escalabilidad:** La implementación de **Pandas 3.0 con backend de PyArrow** permitió reducir la huella de memoria en un ~80% (5.1 MB), garantizando que la aplicación final en Streamlit sea reactiva y eficiente incluso bajo múltiples filtros concurrentes.
* **Integridad de los Datos:** El proceso de limpieza reveló que el estado del vehículo y el kilometraje no siempre coinciden con las etiquetas de origen (ej. vehículos "new" con alto kilometraje). La **imputación contextual** (basada en modelos similares) protegió la distribución de los datos, evitando sesgos que una limpieza simple hubiera introducido.
* **Dinámica de Valor:** El mercado está segmentado por la **utilidad**. Los **Trucks y SUVs** no solo dominan el volumen de anuncios, sino que actúan como activos de refugio de valor, manteniendo precios medianos significativamente superiores a los Sedanes, independientemente del kilometraje.
* **Oportunidad de Negocio:** Se identifica el "punto de quiebre" de rentabilidad en las **100,000 millas**. Por debajo de este umbral, el precio es volátil y sensible a la condición; por encima, el mercado se estabiliza en valores residuales, lo que define estrategias diferenciadas para compradores de presupuesto vs. compradores de valor.

---
**Próximo paso:** Despliegue de estos hallazgos en una interfaz interactiva mediante `app.py`.

In [36]:
# Guarda el dataset limpio para la aplicación
# Usamos compresión 'snappy' para un balance óptimo entre velocidad y peso
df.to_parquet('../data/vehicles_clean.parquet', engine='pyarrow', compression='snappy')
print("✅ Archivo 'vehicles_clean.parquet' generado exitosamente para la App.")

✅ Archivo 'vehicles_clean.parquet' generado exitosamente para la App.


---
---